### **Libraries**

Some of the needed libraries for this project.

In [1]:
import os
import openai
from openai import OpenAI

In [2]:
import pathlib
import textwrap

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") # OpenAI API key

#### **Define a client**

Created an instance of the OpenAI model

In [4]:
# define an instance of OpenAI model
client = OpenAI(api_key=OPENAI_API_KEY)

#### **Audio transcription**

Transcribed the audio file.

In [5]:
audio_file= open("longer_schedule.mp3", "rb")

In [6]:
speech_to_text = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file
)

In [7]:
to_markdown(speech_to_text.text)

> On the 2nd of July, I will be having a masterclass with Julian. While on the 7th, I will be having a synchronized learning with Sik Og. On the 14th of July, by 7pm WET, I will be having another synchronized learning. On the 21st of July, I will be having a synchronized learning between the hours of 7pm to 9pm with Sik Og Analytics. While on the 21st, I will be doing a final presentation with Sik Og Analytics between the hours of 7pm to 9pm.

### **Function calling**

Function to extract all event related data from the provided text.

In [8]:
system_prompt = """
You are an assistant that creates a well crafted schedule from text. 
Extract all events related data from the provided text.
"""

In [9]:
schedule_tool = [
    {
        'type': 'function',
        'function': {
            'name': 'schedule_events',
            'description': 'Extract all events related data from the provided text. If there are multiple events extract all events',
            'parameters': {
                'type': 'object',
                'properties': {
                    'date': {
                        'type': 'array',
                        'items': {
                            'type': 'string',
                            'format': 'date'
                        },
                        'description': 'A list containing the date of every event in the format YYYY-MM-DD'
                    },
                    'start_time': {
                        'type': 'array',
                        'items': {
                            'type': 'string',
                            'format': 'time'
                        },
                        'description': 'The time of the event starts'
                    },
                    'end_time': {
                        'type': 'array',
                        'items': {
                            'type': 'string',
                            'format': 'time'
                        },
                        'description': 'The time of the event ends'
                    },
                    'event': {
                        'type': 'array',
                        'items': {
                            'type': 'string'
                        },
                        'description': 'The name of the event'
                    },
                    'location': {
                        'type': 'array',
                        'items': {
                            'type': 'string'
                        },
                        'description': 'The location of the event'
                    },
                    'event_details': {
                        'type': 'array',
                        'items': {
                            'type': 'string'
                        },
                        'description': 'A description of the event'
                    },
                    'reminder': {
                        'type': 'array',
                        'items': {
                            'type': 'string',
                            'format': 'time'
                        },
                        'description': 'The time before the event to send a reminder'
                    }
                },
                'required': ['date', 'start_time', 'end_time', 'event']
            }
        },
        'instruction': system_prompt
    }
]


#### **Calendar file creation**

Extraction of event data.

In [10]:
#text to extract events from
query = """
On the 2nd of July, I will be having a masterclass with Julian between the hours of 3:00 PM and 4:00 PM WAT. While on the 7th, \
I will be having a synchronized learning with Sik Og. On the 14th of July, by 7pm WET, \
I will be having another synchronized learning. On the 21st of July, I will be having a synchronized \
learning between the hours of 7pm to 9pm with Sik Og Analytics. While on the 21st, I will be doing a final \
presentation with Sik Og Analytics between the hours of 7pm to 9pm.
"""

In [11]:
# user prompt
user_prompt = f"From the text {query} create a calendar-like structure."

calendar_file = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {'role': 'user', 'content': user_prompt}
    ],
    tools=schedule_tool,
    tool_choice='required',
    temperature=0.3
)

In [12]:
print(calendar_file)

ChatCompletion(id='chatcmpl-9iKpK3Yt9hq0P43pMAgnrsKbO1dxT', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_WWKNSj1Crlrr0Jtct0vjXRHI', function=Function(arguments='{"date": ["2023-07-02"], "start_time": ["15:00"], "end_time": ["16:00"], "event": ["Masterclass with Julian"], "location": [], "event_details": [], "reminder": []}', name='schedule_events'), type='function'), ChatCompletionMessageToolCall(id='call_hxDu4tmdLdJwEoLxQwO2BJ1Y', function=Function(arguments='{"date": ["2023-07-07"], "start_time": ["00:00"], "end_time": ["23:59"], "event": ["Synchronized learning with Sik Og"], "location": [], "event_details": [], "reminder": []}', name='schedule_events'), type='function'), ChatCompletionMessageToolCall(id='call_2VxAF9Lpph8DzzJJVu3UBR8x', function=Function(arguments='{"date": ["2023-07-14"], "start_time": ["19:00"], "end_time": ["20:00"]

In [13]:
if(calendar_file.choices[0].message.tool_calls):
  #If tool call was invoked extract the name of the specific function used
  function_name = calendar_file.choices[0].message.tool_calls[0].function.name
  #Print the name of the function called
  #print("called the function: " + function_name)
  # Extract response to user query
  result = calendar_file.choices[0].message.tool_calls[0].function.arguments
else:
  print("didn't call a function")
  result = calendar_file.choices[0].message.content
  
to_markdown(result)

> {"date": ["2023-07-02"], "start_time": ["15:00"], "end_time": ["16:00"], "event": ["Masterclass with Julian"], "location": [], "event_details": [], "reminder": []}

#### **String output to JSON file**

In [14]:
# Used to create a JSON file
import json

In [15]:
# converts string to dictionary
result_dict = json.loads(result)

In [16]:
# 
events_dict = {}
# added the extracted events to the dictionary
events_dict['events'] = [result_dict]
print(events_dict)


{'events': [{'date': ['2023-07-02'], 'start_time': ['15:00'], 'end_time': ['16:00'], 'event': ['Masterclass with Julian'], 'location': [], 'event_details': [], 'reminder': []}]}


In [17]:

# Define the file path
json_file_path = 'events.json'

# Write the result to the JSON file
with open(json_file_path, 'w') as file:
    json.dump(events_dict, file, indent=4)

#### **JSON file to .ics file**

In [20]:
from icalendar import Calendar, Event
from datetime import datetime
from pytz import UTC  # This is required for the 'dt' fields in icalendar

# Path to your JSON file
json_file_path = 'events.json'

# Read and load the JSON data from file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Create an iCalendar file
cal = Calendar()

for event in data['events']:
    # Create an event
    ical_event = Event()
    
    # Assuming the first item in each list is the relevant one
    start_date_str = event['date'][0] + ' ' + event['start_time'][0]
    end_date_str = event['date'][0] + ' ' + event['end_time'][0]
    
    # Convert to datetime objects
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d %H:%M').replace(tzinfo=UTC)
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d %H:%M').replace(tzinfo=UTC)
    
    # Add details to the event
    ical_event.add('summary', event['event'][0])
    ical_event.add('dtstart', start_date)
    ical_event.add('dtend', end_date)
    
    # Optional: Add location and description if available
    if event['location']:
        ical_event.add('location', event['location'][0])
    if event['event_details']:
        ical_event.add('description', event['event_details'][0])
    
    # Add the event to the calendar
    cal.add_component(ical_event)

# Save the iCalendar file
file_name = 'events.ics'
with open(file_name, 'wb') as f:
    f.write(cal.to_ical())

print(f"iCalendar file '{file_name}' created successfully.")

iCalendar file 'events.ics' created successfully.
